# 🧪 EXPERIMENTO 2: Chain-of-Thought Adversarial (Spectral Collapse)

## Objetivo
**Provar que nem todo CoT é benéfico: Pensamentos "instáveis" destroem a memória.**

## Hipótese (Spectral Collapse)
- O passo de discretização $\Delta$ depende do input ($x_t$).
- **Adversarial CoT**: Inputs projetados para gerar $\Delta$ grande.
- **Consequência**: $\bar{A} = \exp(\Delta A)$ decai muito rápido (ρ próximo de 0).
- **Resultado Esperado**: A accuracy cai drasticamente comparado a um CoT "benigno" de mesmo tamanho.

---
**Runtime: GPU recomendado, ~3 min**

In [ ]:
#@title 1️⃣ Setup e Imports
%%capture
!pip install torch numpy scipy matplotlib seaborn pandas

import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json
import shutil
from google.colab import files

sns.set_theme(style="whitegrid")
os.makedirs('exp2_results', exist_ok=True)

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device: {DEVICE}")

In [ ]:
#@title 2️⃣ Simulação do Mecanismo de Discretização

# No Mamba: delta = softplus(Linear(x) + bias)
# Vamos simular esse mecanismo

class MambaDiscretizer(torch.nn.Module):
    def __init__(self, d_model):
        super().__init__()
        self.linear = torch.nn.Linear(d_model, 1)
        self.softplus = torch.nn.Softplus()
        
        # Inicialização para gerar deltas razoáveis (média ~0.01)
        torch.nn.init.normal_(self.linear.weight, std=0.01)
        torch.nn.init.constant_(self.linear.bias, -2.5)
    
    def forward(self, x):
        # x: [batch, d_model]
        # delta: [batch, 1]
        return self.softplus(self.linear(x))

# Criar modelo simulado
d_model = 768
discretizer = MambaDiscretizer(d_model).to(DEVICE)

print("Mecanismo de discretização criado.")

In [ ]:
#@title 3️⃣ Gerar CoT: Benigno vs Adversarial

# Benigno: Inputs que mantêm delta pequeno (preserva memória)
# Adversarial: Inputs que explodem delta (apagam memória)

def generate_cot_inputs(n_steps, mode='benign'):
    inputs = torch.randn(n_steps, d_model, device=DEVICE)
    
    if mode == 'benign':
        # Otimizar inputs para minimizar delta
        inputs.requires_grad = True
        optimizer = torch.optim.Adam([inputs], lr=0.1)
        for _ in range(50):
            deltas = discretizer(inputs)
            loss = torch.mean(deltas) # Minimizar delta
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        return inputs.detach()
        
    elif mode == 'adversarial':
        # Otimizar inputs para MAXIMIZAR delta (ataque)
        inputs.requires_grad = True
        optimizer = torch.optim.Adam([inputs], lr=0.1)
        for _ in range(50):
            deltas = discretizer(inputs)
            loss = -torch.mean(deltas) # Maximizar delta
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        return inputs.detach()
    
    else: # Random
        return inputs

# Teste
cot_len = 50
x_benign = generate_cot_inputs(cot_len, 'benign')
x_adv = generate_cot_inputs(cot_len, 'adversarial')

with torch.no_grad():
    d_benign = discretizer(x_benign).mean().item()
    d_adv = discretizer(x_adv).mean().item()

print(f"Delta Médio (Benigno):     {d_benign:.4f} (Memória preservada)")
print(f"Delta Médio (Adversarial): {d_adv:.4f}  (Memória apagada)")
print(f"Fator de Aceleração:       {d_adv/d_benign:.1f}x")

In [ ]:
#@title 4️⃣ Simulação de Memória no Tempo

# Parâmetros do sistema
d_state = 16
A_log = torch.randn(d_model, d_state, device=DEVICE) - 1.5
A = -torch.exp(A_log) # Matriz A do Mamba

def simulate_memory_decay(inputs):
    """Simula o decaimento da informação passando pelo CoT"""
    # Estado inicial h0 (informação original) = 1.0
    current_memory = 1.0
    memory_trace = [1.0]
    
    deltas = discretizer(inputs)
    
    # Evolução passo a passo
    for delta in deltas:
        d = delta.item()
        # Raio espectral neste passo: max(|exp(delta * A)|)
        # Para A diagonal negativo: exp(delta * max(A))
        # Mas queremos o efeito médio no estado. 
        # Simplificação: memory *= exp(delta * A_mean)
        
        # Usando o maior autovalor (pior caso de retenção, melhor caso de estabilidade)
        # Como A é negativo, quanto maior delta, menor o valor resultante (mais próximo de 0)
        # rho = exp(delta * A_val) onde A_val é tipicamente -1 a -0.1
        
        rho_step = np.exp(d * -0.5) # Assumindo um autovalor típico de -0.5
        current_memory *= rho_step
        memory_trace.append(current_memory)
        
    return memory_trace

# Simular
trace_benign = simulate_memory_decay(x_benign)
trace_adv = simulate_memory_decay(x_adv)

print(f"Memória Final (Benigno):     {trace_benign[-1]:.2%}")
print(f"Memória Final (Adversarial): {trace_adv[-1]:.2%}")

In [ ]:
#@title 5️⃣ GRÁFICO 1: Colapso Espectral

plt.figure(figsize=(10, 6))
plt.plot(trace_benign, label='CoT Benigno (Focado)', color='blue', linewidth=2)
plt.plot(trace_adv, label='CoT Adversarial (Confuso)', color='red', linewidth=2)
plt.axhline(0.5, color='gray', linestyle='--', alpha=0.5, label='Limite Útil')

plt.title('Colapso de Memória: Benigno vs Adversarial', fontsize=14)
plt.xlabel('Passos de Raciocínio (Tokens)', fontsize=12)
plt.ylabel('Retenção de Informação', fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)

plt.savefig('exp2_results/spectral_collapse.png', dpi=150)
plt.show()

In [ ]:
#@title 6️⃣ EXPERIMENTO COMPLETO: Impacto na Accuracy

# Variando o tamanho do CoT
cot_lengths = [10, 20, 50, 100, 200]
results = []

print("Rodando simulação para diferentes tamanhos de CoT...")

for length in cot_lengths:
    # Gerar CoTs
    xb = generate_cot_inputs(length, 'benign')
    xa = generate_cot_inputs(length, 'adversarial')
    
    # Medir retenção final
    mb = simulate_memory_decay(xb)[-1]
    ma = simulate_memory_decay(xa)[-1]
    
    # Converter para Accuracy (usando a função do Exp 1)
    acc_b = 1 / (1 + np.exp(-10 * (mb - 0.01)))
    acc_a = 1 / (1 + np.exp(-10 * (ma - 0.01)))
    
    results.append({'length': length, 'type': 'Benign', 'accuracy': acc_b})
    results.append({'length': length, 'type': 'Adversarial', 'accuracy': acc_a})

df_res = pd.DataFrame(results)

# Gráfico de Barras
plt.figure(figsize=(10, 6))
sns.barplot(data=df_res, x='length', y='accuracy', hue='type', palette={'Benign': 'blue', 'Adversarial': 'red'})

plt.title('Impacto do CoT Adversarial na Acurácia Final', fontsize=14)
plt.xlabel('Tamanho do CoT (tokens)', fontsize=12)
plt.ylabel('Accuracy na Resposta', fontsize=12)
plt.axhline(0.5, color='red', linestyle='--', alpha=0.5)

plt.savefig('exp2_results/accuracy_impact.png', dpi=150)
plt.show()

In [ ]:
#@title 7️⃣ Conclusão e Download

mean_diff = df_res[df_res['type']=='Benign']['accuracy'].mean() - df_res[df_res['type']=='Adversarial']['accuracy'].mean()

print("="*40)
print("🔐 RESULTADOS EXPERIMENTO 2")
print("="*40)
print(f"Diferença Média de Accuracy: {mean_diff:.2%}")

if mean_diff > 0.3:
    print("✅ HIPÓTESE VALIDADA: CoT Adversarial causa colapso severo.")
else:
    print("⚠️ INCONCLUSIVO.")

# JSON
final_data = {
    'experiment': 'Adversarial CoT',
    'mean_accuracy_drop': mean_diff,
    'hypothesis_validated': bool(mean_diff > 0.3)
}
with open('exp2_results/results.json', 'w') as f:
    json.dump(final_data, f)

shutil.make_archive('exp2_cot_adversarial', 'zip', 'exp2_results')
files.download('exp2_cot_adversarial.zip')